In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
import re

In [117]:
class Segment():
    def __init__(self, info_lines, data_lines, number):
        self.number = number
        self.info = self.get_info(info_lines)
        self.data = self.get_data(data_lines)

    def get_data(self, lines):
        data = [line.strip().split(', ') for line in lines[1:] if line.strip()]
        return pd.DataFrame(data, columns=['Potential/V', 'Current/A', 'Charge/C']).astype({'Potential/V': float, 'Current/A': float, 'Charge/C': float})

    def get_info(self, lines):
        Eps, ips, Ahs = [], [], []
        for line in lines:
            if line.startswith('Ep'):
                Eps.append(line.split('=')[1].strip())
            elif line.startswith('ip'):
                ips.append(line.split('=')[1].strip())
            elif line.startswith('Ah'):
                Ahs.append(line.split('=')[1].strip())

        segment_info = {
            'Segment': self.number,
            'peaks': [{'Ep':e, 'ip':i, 'Ah':a} for e, i, a in zip(Eps, ips, Ahs)] if len(Eps)*len(ips)*len(Ahs) != 0 else None,
        }

        return segment_info

        

class CyclicVoltammetry():
    def __init__(self, file_name):
        self.name = file_name.split('.')[0]
        self.text = self.read_file(file_name)
        self.inifo = self.initialize()
        self.segifo = self.init_segments()

    def read_file(self, file_name):
        with open(file_name) as f:
            lines = f.readlines()
        return lines
    
    def initialize(self):
        initial_info = {
            'Name': self.name,
            'Init E (V)': None,
            'High E (V)': None,
            'Low E (V)': None,
            'Scan Rate (V/s)': None,
            'Segments': None,
            'Sample Interval (V)': None,
            'Sensitivity (A/V)': None,
        }

        combined_text = "\n".join(self.text).split('Segment 1:')[0]

        init_e_pattern = r"Init E \(V\) = ([\-\d\.]+)"
        high_e_pattern = r"High E \(V\) = ([\-\d\.]+)"
        low_e_pattern = r"Low E \(V\) = ([\-\d\.]+)"
        scan_rate_pattern = r"Scan Rate \(V/s\) = ([\-\d\.e]+)"
        segments_pattern = r"Segment = (\d+)"
        sample_interval_pattern = r"Sample Interval \(V\) = ([\-\d\.e]+)"
        sensitivity_pattern = r"Sensitivity \(A/V\) = ([\-\d\.e\-]+)"

        initial_info['Init E (V)'] = re.search(init_e_pattern, combined_text).group(1)
        initial_info['High E (V)'] = re.search(high_e_pattern, combined_text).group(1)
        initial_info['Low E (V)'] = re.search(low_e_pattern, combined_text).group(1)
        initial_info['Scan Rate (V/s)'] = re.search(scan_rate_pattern, combined_text).group(1)
        initial_info['Segments'] = int(re.search(segments_pattern, combined_text).group(1))
        initial_info['Sample Interval (V)'] = re.search(sample_interval_pattern, combined_text).group(1)
        initial_info['Sensitivity (A/V)'] = re.search(sensitivity_pattern, combined_text).group(1)

        return initial_info

    def init_segments(self):
        segments = []
        
        for i, line in enumerate(self.text):
            if line.startswith('Segment 1:'):
                segments_text = self.text[i:]
                break
        else:
            raise ValueError('Find segment text failed')
        
        for i, line in enumerate(segments_text):
            if line.startswith('Potential/V, Current/A, Charge/C'):
                segments_info_text = segments_text[:i]
                segments_data_text = segments_text[i+1:]
                break
        
        segments_info, segments_data = [], []
        segment_info_index, segment_data_index = [], []

        for i, line in enumerate(segments_info_text):
            if line.startswith('Segment '):
                segment_info_index.append(i)

        for i, line in enumerate(segments_data_text):
            if line.startswith('Segment '):
                segment_data_index.append(i)

        for i in range(len(segment_info_index)):
            if i == len(segment_info_index) - 1:
                segments_info.append(segments_info_text[segment_info_index[i]:])
            else:
                segments_info.append(segments_info_text[segment_info_index[i]:segment_info_index[i+1]])

        for i in range(len(segment_data_index)):
            if i == len(segment_data_index) - 1:
                segments_data.append(segments_data_text[segment_data_index[i]:])
            else:
                segments_data.append(segments_data_text[segment_data_index[i]:segment_data_index[i+1]])

        for i, j in zip(segments_info, segments_data):
            segments.append(Segment(i, j, len(segments) + 1))

        return segments
    
    
    def get_all_data(self):
        all_data = [segment.data for segment in self.segifo]
        return all_data

    def get_segment_data(self, segment_number):
        if segment_number <= len(self.segifo):
            return self.segifo[segment_number - 1].data
        else:
            raise ValueError("Segment number out of range")

    def get_all_info(self):
        all_info = [segment.info for segment in self.segifo]
        return all_info

    def get_segment_info(self, segment_number):
        if segment_number <= len(self.segifo):
            return self.segifo[segment_number - 1].info
        else:
            raise ValueError("Segment number out of range")
        
    def get_initial_info(self):
        return self.inifo


In [118]:
file_names = ['blank1.txt', 
              'blank2.txt',
              'blank5.txt']

In [119]:
cvs = [CyclicVoltammetry(file_name) for file_name in file_names]

In [120]:
cvs[0].get_initial_info()

{'Name': 'blank1',
 'Init E (V)': '0.65',
 'High E (V)': '1.1',
 'Low E (V)': '-0.3',
 'Scan Rate (V/s)': '0.1',
 'Segments': 5,
 'Sample Interval (V)': '0.01',
 'Sensitivity (A/V)': '1e-5'}

In [122]:
cvs[0].name

'blank1'